# Importing modules

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import itertools
from scipy import stats as st
import statsmodels as sm
import statsmodels.api as sma
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import h5py
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
import statisticalTools as sT
from sklearn.neighbors.kde import KernelDensity
import properscoring as ps
warnings.filterwarnings("ignore")

%matplotlib inline 
#%matplotlib 
plt.figure(figsize=(15, 6))

C:\Users\SERRA_R\AppData\Local\Continum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Reading data and extracting information

In [3]:
#Read data
file_name = 'data/GENS_test_AllLevels3'+'.hdf'
dt = pd.read_hdf(file_name, 'Wind_vector')
#Sort the dataset in function of date
dt = dt.set_index('Timestamp')
dt = dt.sort_index()

In [4]:
#Read data
file_name = 'data/gens-a_3'+'.hdf'
DT = pd.read_hdf(file_name, 'Wind_vector')
DT = DT.set_index('Timestamp')
DT = DT.sort_index()

In [ ]:
#Selects the time steps presents on both datasets.
ind0 = dt.index.unique()
ind1 = DT.index.unique()
ind = set(ind0) & set(ind1)

dtt = dt.loc[ind]
DTT = DT.loc[ind]

dtc = dtt.where(dtt.Forecast == '000').where(dtt.NForecast == 999).dropna(how='all') #Observation - Most probable forecast
dtd = dtt.where(dtt.Forecast == '006').where(dtt.NForecast == 999).dropna(how='all') #Forecast - Most probable forecast

In [ ]:
#Selects the latitude, longitudes and levels steps presents on both datasets.
lats0 = dtt.where(dtt.NForecast == 999).dropna(how = 'all').lat.unique()
longs0 = dtt.where(dtt.NForecast == 999).dropna(how = 'all').long.unique()
levels0 = dtt.where(dtt.NForecast == 999).dropna(how = 'all').level.unique()

lats1 = DTT.lat.unique()
longs1 = DTT.long.unique()
levels1 = DTT.level.unique()

lats = list(set(lats0) & set(lats1))
longs = list(set(longs0) & set(longs1))
levels = list(set(levels0) & set(levels1))

#shows the latitudes, longitudes and levels presents in both datasets
print(lats)
print(longs)
print(levels)

In [ ]:
#Selecting data to compare

dx = dt
dt = dtt
dt = dt.sort_index()
dt = dt.where(dt.Forecast == '000')
dt = dt.where(dt.NForecast == 999)
dy = dt.dropna()
#dt = dt.ix[dt.index[1]]

dt = dtt
dt = dt.sort_index()
dt = dt.where(dt.Forecast == '006')
dt = dt.where(dt.NForecast == 999)
dyf = dt.dropna()

dt = dx

In [ ]:

oi = 0

dfp = DTT
dyp = dy
dyfp = dyf

dfp = dfp.where(dfp.lat == lats[oi] )
dfp = dfp.where(dfp.long == longs[oi] )
dfp = dfp.where(dfp.level == levels[oi] )
dfp = dfp.dropna(how = 'all')

dyp = dyp.where(dyp.lat == lats[oi] )
dyp = dyp.where(dyp.long == longs[oi]) 
dyp = dyp.where(dyp.level == levels[oi]) 
dyp = dyp.dropna(how='all')

dyfp = dyfp.where(dyfp.lat == lats[oi]) 
dyfp = dyfp.where(dyfp.long == longs[oi]) 
dyfp = dyfp.where(dyfp.level == levels[oi]) 
dyfp = dyfp.dropna(how='all')

# Verify results using punctual metric

In [ ]:
#defining variables and parameters
count_loop = 0

param = 'Wy'

my_para = []
my_para0 = []
count_loop = 0
Result = []
Result0 = []
Result1 = []
Result2 = []

ind0 = dtc.index.unique()
ind1 = DTT.index.unique()
ind = set(ind0) & set(ind1)

dtc = dtc.loc[ind]
DTT = DTT.loc[ind]
dtd = dtd.loc[ind]

lats = DTT.lat.unique()
longs = DTT.long.unique()
levels = DTT.level.unique()

#if one wants to test fast if it works
#levels = levels[6]
#lats = lats[0:1]
#longs = longs[0:1]

In [ ]:

t_t = time.time()
for i in levels:
    count_loop += 1
    print(count_loop,'of',len(levels), end = ' ||| ')
    for j in range(len(lats)):
        for k in range(len(longs)):
            temp = dtc.where(dtc.level == i)
            temp = temp.where(temp.lat == lats[j]).where(temp.long == longs[k]).dropna(how='all')
                
            temp0 = DTT.where(DTT.level == i)
            temp0 = temp0.where(temp0.lat == lats[j]).where(temp0.long == longs[k]).dropna(how='all')
            
            temp1 = dtd.where(dtd.level == i)
            temp1 = temp1.where(temp1.lat == lats[j]).where(temp1.long == longs[k]).dropna(how='all')
            
            for timer in ind:
                try:
                    Obs = temp.loc[timer]
                    Nforc = temp0.loc[timer]
                    ONEforc = temp1.loc[timer]

                    #param = 'Wy'
                    nforc = Nforc[param]
                    nforc = nforc[1:]
                    oneforc = ONEforc[param]
                    obs = Obs[param]               
                    nforc = np.array(sorted(nforc))
                    #obs = np.ones([len(nforc)])*obs

                    #Ensemble Forecast
                    Result += [ps.crps_ensemble(obs, nforc)]
                    
                    #Gaussian
                    mu, sigma = norm.fit(nforc)
                    Result0 += [ps.crps_gaussian(obs, mu=mu, sig=sigma)]
                    #Gaussian 1
                    Result2 += [ps.crps_gaussian(obs, mu=oneforc, sig=sigma)]
                    #KDE
                    KDEpdf = gaussian_kde(nforc)
                    xmean = np.abs(np.mean(nforc))
                    xmin = np.min(nforc)-.2*xmean
                    xmax = np.max(nforc)+.2*xmean
                    x = np.linspace(xmin,xmax,1000)
                    Result1 += [ps.crps_ensemble(obs, x, KDEpdf(x))]

                    
                    sizet += 1
                except:
                    a = 0
                    #print('one error')
            #print(str(time.time()-t_t)+ " have passed since the start of the calcul.")  
    print(str( (time.time()-t_t)*(len(levels)-count_loop)/count_loop)+ "s to finish")  
print(str(time.time()-t_t)+ " seconds to calculate")            

CPRS_mean = np.mean(Result)
CPRS_mean0 = np.mean(Result0)
CPRS_mean1 = np.mean(Result1)
CPRS_mean2 = np.mean(Result2)

CPRS_sum = np.sum(Result)
CPRS_sum0 = np.sum(Result0)
CPRS_sum1 = np.sum(Result1)
CPRS_sum2 = np.sum(Result2)

print('CPRS_mean Ensemble forecast:',CPRS_mean)
print('CPRS_mean Gaussian:',CPRS_mean0)
print('CPRS_mean KDE:',CPRS_mean1)
print('CPRS_mean Gaussian 1:',CPRS_mean2)

print('')

print('CPRS_sum Ensemble forecast:',CPRS_sum)
print('CPRS_sum Gaussian:',CPRS_sum0)
print('CPRS_sum KDE:',CPRS_sum1)